In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read csv

In [ ]:
# df=pd.read_csv('/kaggle/input/subject3/10003_26038.csv')
# df=pd.read_csv('/kaggle/input/subject-5/10015_10063.csv')
# df=pd.read_csv('/kaggle/input/subject-9/12808_17752.csv')
# df=pd.read_csv('/kaggle/input/testannotation/10000_17728.csv')
df=pd.read_csv('/kaggle/input/9955-18184/9955_18184.csv')
# df=pd.read_csv('/kaggle/input/9931-17578/9931_17578.csv')
# df=pd.read_csv('/kaggle/input/subject-5/10015_10063.csv')


print(df.isna().sum())
df_og = df

In [ ]:
n = len(df)
n

## remove rows with "sleep stage ?"

In [ ]:
df = df_og

In [ ]:
# n = len(df)
df = df.iloc[0:8033528]
len(df)

In [ ]:
df.columns

In [ ]:
print(df.isna().sum())

In [ ]:
import pandas as pd


# Print maximum values of all columns
print("Maximum values of all columns:")
print(df.max())

# Print minimum values of all columns
print("\nMinimum values of all columns:")
print(df.min())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Step 1: Create a balanced dataset
# anomaly_rows = df[df['anomalies'] == 1].sample(10000, random_state=42)
# non_anomaly_rows = df[df['anomalies'] == 0].sample(10000, random_state=42)

# anomaly_rows = wavelet_features_df[wavelet_features_df['anomalies'] == 1].sample(10000, random_state=42)
# non_anomaly_rows = wavelet_features_df[wavelet_features_df['anomalies'] == 0].sample(10000, random_state=42)
anomaly_rows = scaled_n1_df[scaled_n1_df['anomalies'] == 1].sample(4000, random_state=42)
non_anomaly_rows = scaled_n1_df[scaled_n1_df['anomalies'] == 0].sample(4000, random_state=42)

# Combine the anomaly and non-anomaly rows
balanced_df = pd.concat([anomaly_rows, non_anomaly_rows])

# Step 2: Prepare the data
X = balanced_df.drop(columns=['anomalies'])  # Features (drop the target column)
y = balanced_df['anomalies']  # Target variable

# Split the data into training and testing sets (optional, but recommended)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: Extract the Gini importance for each feature
importances = clf.feature_importances_

# Create a DataFrame to display the feature importances
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Gini Importance': importances
}).sort_values(by='Gini Importance', ascending=False)

# Display the feature importances
print(feature_importance_df)

# Create a SHAP explainer
# explainer = shap.Explainer(clf, X)

# # Calculate SHAP values (SHAP interaction values can also be calculated for more insights)
# shap_values = explainer(X)

# # Plot summary plot to show feature importance
# shap.summary_plot(shap_values, X)

# # You can also plot feature importance bar plot from SHAP values
# shap.summary_plot(shap_values, X, plot_type="bar")


# EDA (anomaly list)

In [ ]:
# df1=pd.read_csv('/kaggle/input/subject3/10003_26038 (1).tsv',sep='\t')
# df1=pd.read_csv('/kaggle/input/subject-5/10015_10063.tsv',sep='\t')
# df1=pd.read_csv('/kaggle/input/subject-9/12808_17752.tsv',sep='\t')
# df1=pd.read_csv('/kaggle/input/testfinal/10000_17728 (1).tsv',sep='\t')
df1=pd.read_csv('/kaggle/input/9955-18184-tsv/9955_18184.tsv',sep='\t')
# df1=pd.read_csv('/kaggle/input/9931-17578-tsv/9931_17578.tsv',sep='\t')
# df1=pd.read_csv('/kaggle/input/subject-5/10015_10063.tsv',sep='\t')



print(set(df1['description'].values))

In [ ]:
# anomalies_list = [
#     'Central Apnea', 'Grit Teeth or Chew (5 seconds)', 'Breath Hold (10 seconds)',
#     'Simulate Snore or Hum (5 seconds)','Obstructive Apnea','Obstructive Hypopnea',
#     'Looking around', 'Eyes Up and Down X5',
#     'Mixed Apnea', 'Oral Breathing (10 seconds)', 'Oxygen Desaturation',
#     'Flex Right Foot', 'So', 'Flex Left Foot', 'Eye Blinks X5', 'Bruxism', 'Eyes Left and Right X5'
# ]
anomalies_list = ['Oxygen Desaturation']
# anomalies_list = ['Obstructive Apnea','Mixed Apnea','Central Apnea']

# Assume df1 is your DataFrame containing the annotations
# Filter annotations dataframe to include only anomalies
anomalies_df = df1[df1['description'].isin(anomalies_list)]

# Initialize the binary data list with zeros
# binary_data1 = [0] * len(smoothed_df)
binary_data1 = [0] * len(df)

# Iterate over each row in anomalies_df
for _, row in anomalies_df.iterrows():
    start_row = int(row['onset'] * 256)
    end_row = int((row['onset'] + row['duration']) * 256)
    # Mark intervals as 1
    for i in range(start_row, end_row + 1):
        if i < len(binary_data1):
            binary_data1[i] = 1

# Add the binary data as a new column to smoothed_df
# smoothed_df['anomalies'] = binary_data1
df['anomalies'] = binary_data1

# Display the updated DataFrame
print(df.head())

In [ ]:
print(binary_data1.count(1))

## Grouping into sleep stages

In [ ]:
# Define the list of sleep stages
sleep_stages = ['Sleep stage W', 'Sleep stage N1', 'Sleep stage N2', 'Sleep stage N3','Sleep stage R']

# Initialize empty DataFrames for each sleep stage
w_df = pd.DataFrame()
n1_df = pd.DataFrame()
n2_df = pd.DataFrame()
n3_df = pd.DataFrame()
r_df = pd.DataFrame()

# Create a dictionary to map sleep stages to their respective DataFrames
sleep_stage_dfs = {
    'Sleep stage W': w_df,
    'Sleep stage N1': n1_df,
    'Sleep stage N2': n2_df,
    'Sleep stage N3': n3_df,
    'Sleep stage R':r_df
}

# Filter the annotations dataframe to include only rows for sleep stages
sleep_stage_df = df1[df1['description'].isin(sleep_stages)]

# Iterate over each row in sleep_stage_df
for _, row in sleep_stage_df.iterrows():
    start_row = int(row['onset'] * 256)  # Calculate the start row in smoothed_df
    end_row = int((row['onset'] + row['duration']) * 256)  # Calculate the end row in smoothed_df
    stage = row['description']  # Get the sleep stage (e.g., 'Sleep Stage W')
    
    # Get the corresponding rows from smoothed_df and append to the respective DataFrame
    stage_rows = df.iloc[start_row:end_row + 1]
    sleep_stage_dfs[stage] = pd.concat([sleep_stage_dfs[stage], stage_rows])




In [ ]:
# Now w_df, n1_df, n2_df, and n3_df will contain the corresponding rows from smoothed_df
w_df = sleep_stage_dfs['Sleep stage W']
n1_df = sleep_stage_dfs['Sleep stage N1']
n2_df = sleep_stage_dfs['Sleep stage N2']
n3_df = sleep_stage_dfs['Sleep stage N3']
r_df =  sleep_stage_dfs['Sleep stage R']

# Optionally, display the DataFrames for each sleep stage
print("W Stage DataFrame:")
print(w_df)
print("\nN1 Stage DataFrame:")
print(n1_df)
print("\nN2 Stage DataFrame:")
print(n2_df)
print("\nN3 Stage DataFrame:")
print(n3_df)
print("\nR Stage DataFrame:")
print(r_df)

In [ ]:
w_df.columns,len(w_df)

# minimum SpO2

In [ ]:
print("Minimum SpO2 value for sleep stage W as an anomaly:")
min_SpO2_w = w_df[w_df['anomalies'] == 1]['SpO2'].min()
print(min_SpO2_w,w_df['SpO2'].min())

print("Minimum SpO2 value for sleep stage N1 as an anomaly:")
min_SpO2_n1 = n1_df[n1_df['anomalies'] == 1]['SpO2'].min()
print(min_SpO2_n1,n1_df['SpO2'].min())

print("Minimum SpO2 value for sleep stage N2 as an anomaly:")
min_SpO2_n2 = n2_df[n2_df['anomalies'] == 1]['SpO2'].min()
print(min_SpO2_n2,n2_df['SpO2'].min())

print("Minimum SpO2 value for sleep stage N3 as an anomaly:")
min_SpO2_n3 = n3_df[n3_df['anomalies'] == 1]['SpO2'].min()
print(min_SpO2_n3,n3_df['SpO2'].min())

print("Minimum SpO2 value for sleep stage R as an anomaly:")
min_SpO2_r = r_df[r_df['anomalies'] == 1]['SpO2'].min()
print(min_SpO2_r,r_df['SpO2'].min())

In [ ]:
w_df = w_df[w_df['SpO2'] >= max(70,min_SpO2_w)].reset_index(drop=True)
n1_df = n1_df[n1_df['SpO2'] >= min_SpO2_n1].reset_index(drop=True)
n2_df = n2_df[n2_df['SpO2'] >= min_SpO2_n2].reset_index(drop=True)
n3_df = n3_df[n3_df['SpO2'] >= min_SpO2_n3].reset_index(drop=True)
r_df = r_df[r_df['SpO2'] >= min_SpO2_r].reset_index(drop=True)

# w_df = w_df[w_df['SpO2'] >= 78.58853130960594].reset_index(drop=True)
# n1_df = n1_df[n1_df['SpO2'] >= 0.0040582768556589].reset_index(drop=True)
# n2_df = n2_df[n2_df['SpO2'] >= 86.44941357899437].reset_index(drop=True)
# n3_df = n3_df[n3_df['SpO2'] >= 92.34203157339394].reset_index(drop=True)
# r_df = r_df[r_df['SpO2'] >= 92.34203157339394].reset_index(drop=True)

In [ ]:
df = df[df['SpO2'] >= 70].reset_index(drop=True)

In [ ]:
# len(df)
total = len(w_df)+len(n1_df)+len(n2_df)+len(n3_df)+len(r_df)
print(total,len(df))                                        

In [ ]:
print("Percentage of W stage:",((len(w_df)/(len(w_df)+len(n1_df)+len(n2_df)+len(n3_df)+len(r_df)))*100))
print("Percentage of N1 stage:",((len(n1_df)/(len(w_df)+len(n1_df)+len(n2_df)+len(n3_df)+len(r_df)))*100))
print("Percentage of N2 stage:",((len(n2_df)/(len(w_df)+len(n1_df)+len(n2_df)+len(n3_df)+len(r_df)))*100))
print("Percentage of N3 stage:",((len(n3_df)/(len(w_df)+len(n1_df)+len(n2_df)+len(n3_df)+len(r_df)))*100))
print("Percentage of R stage:",((len(r_df)/(len(w_df)+len(n1_df)+len(n2_df)+len(n3_df)+len(r_df)))*100))

In [ ]:
len(n3_df),n3_df.columns

In [ ]:

# w_df = w_df.drop("EtCO2", axis='columns')
# n1_df = n1_df.drop("EtCO2", axis='columns')
# n2_df = n2_df.drop("EtCO2", axis='columns')
# n3_df = n3_df.drop("EtCO2", axis='columns')
# r_df = r_df.drop("EtCO2", axis='columns')

In [ ]:
df.shape
df.columns

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming 'anomalies' is your ground truth column in wavelet_features_df
# and 'anomaly' is the predicted label from the K-means clustering step.

# Step 1: Map the 'anomalous' and 'non-anomalous' values to binary labels for comparison
# Ground truth: 'anomalous' should map to 1, 'non-anomalous' to 0
wavelet_features_df['ground_truth'] = wavelet_features_df['anomalies'].map(lambda x: 1 if x == 'anomalous' else 0)
wavelet_features_df['predicted'] = wavelet_features_df['anomaly'].map(lambda x: 1 if x == 'anomalous' else 0)

# Step 2: Calculate evaluation metrics
accuracy = accuracy_score(wavelet_features_df['ground_truth'], wavelet_features_df['predicted'])
precision = precision_score(wavelet_features_df['ground_truth'], wavelet_features_df['predicted'])
recall = recall_score(wavelet_features_df['ground_truth'], wavelet_features_df['predicted'])
f1 = f1_score(wavelet_features_df['ground_truth'], wavelet_features_df['predicted'])
conf_matrix = confusion_matrix(wavelet_features_df['ground_truth'], wavelet_features_df['predicted'])
balanced_accuracy = balanced_accuracy_score(wavelet_features_df['ground_truth'], wavelet_features_df['predicted'])

# Print the evaluation results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
tn, fp, fn, tp = confusion_matrix(wavelet_features_df['ground_truth'], wavelet_features_df['predicted']).ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)
print("Balanced Accuracy: ",(specificity+recall)/2)
print("Confusion Matrix:\n", conf_matrix)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix

accuracies = []
for percent in range(30,90,10):
    percent /= 100
    percent_index = int(percent * len(wavelet_features_df))
    
    # Splitting the data 
    train_df = wavelet_features_df.iloc[:percent_index]
    test_df = wavelet_features_df.iloc[percent_index:]
    
    # Separate the minority and majority classes in the training set
    anomalies = train_df[train_df['anomalies'] == 1]
    non_anomalies = train_df[train_df['anomalies'] == 0]
    
    # Downsampling the majority class to match the minority class size
    non_anomalies_downsampled = resample(non_anomalies, 
                                         replace=False,  # sample without replacement
                                         n_samples=len(anomalies),  # match number of anomalies
                                         random_state=42)  # fixed seed for reproducibility
    
    # Create a balanced training dataset
    balanced_train_df = pd.concat([anomalies, non_anomalies_downsampled])
    
    # Separate features and target for training and testing
    # X_train = balanced_train_df.drop(columns=['anomalies'])
    # y_train = balanced_train_df['anomalies']
    # X_test = test_df.drop(columns=['anomalies'])
    # y_test = test_df['anomalies']
    
    # Split data into features (X) and target (y)
    X = wavelet_features_df.drop(columns=['anomalies'])
    y = wavelet_features_df['anomalies']
    
    # # Split data into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-percent, random_state=42)
    
    # Train an SVM model
    svm_model = SVC(kernel='rbf', random_state=42)
    svm_model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = svm_model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Calculate specificity
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    # Print results
    # print("Accuracy:", accuracy)
    print("Balanced Accuracy:", balanced_accuracy)
    # print("Specificity:", specificity)
    accuracies.append(balanced_accuracy)
    
print(max(accuracies))


In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import fft

# Assuming smoothed_n2_df is your DataFrame with all the relevant columns
# List of columns to process
columns_to_process = ['EOG LOC-M2', 'EOG ROC-M1', 'EMG Chin2-Chin1', 'EEG F3-M2', 'EEG F4-M1', 'EEG C3-M2', 
                      'EEG C4-M1', 'EEG O1-M2', 'EEG O2-M1', 'EEG CZ-O1', 'EMG LLeg-RLeg', 'ECG EKG2-EKG', 
                      'Snore', 'Resp PTAF', 'Resp Airflow', 'Resp Thoracic', 'Resp Abdominal', 'SpO2', 'Rate',  'Capno', 'Resp Rate', 'C-flow', 'Tidal Vol', 'Pressure']

# Parameters
segment_size = 1280  # 10 seconds segment (1280 rows)
anomaly_threshold = 0.10  # 10% anomaly threshold

# Lists to store results
feature_rows = []

# Loop through the data and segment it for each column
for start_idx in range(0, len(df), segment_size):
    end_idx = start_idx + segment_size
    
    # Ensure segment does not go out of bounds
    if end_idx > len(df):
        break
    
    # Store features for each column in this segment
    segment_features = {}
    
    # Loop through each column
    for col in columns_to_process:
        # Extract the data and annotations for the current column
        segment = df[col].iloc[start_idx:end_idx].values
        segment_annotations = df['anomalies'].iloc[start_idx:end_idx].values

        # Check if the segment contains more than 10% anomalies
        if np.mean(segment_annotations) > anomaly_threshold:
            segment_label = 1  # Anomalous
        else:
            segment_label = 0  # Non-anomalous

        # Apply FFT to the segment
        fft_segment = fft(segment)
        
        # Compute the frequency bins
        n = len(segment)
        freqs = np.fft.fftfreq(n)

        # Calculate the amplitude spectrum
        amplitude_segment = np.abs(fft_segment)
        # print(amplitude_segment.shape)

        # Extract the dominant frequency (excluding zero frequency)
        dominant_freq = freqs[np.argmax(amplitude_segment[1:n//2])]

        # Store features: dominant frequency, mean amplitude, and mean value
        segment_features[f'{col}_dominant_freq'] = dominant_freq
        segment_features[f'{col}_mean_amplitude'] = np.mean(amplitude_segment)  # Mean of the amplitude
        segment_features[f'{col}_mean_value'] = np.mean(segment)  # Mean of the original segment

    # Add the label
    segment_features['label'] = segment_label
    
    # Append to feature rows
    feature_rows.append(segment_features)

# Create a DataFrame from the extracted features
feature_df = pd.DataFrame(feature_rows)

# Display the resulting DataFrame
# print(feature_df)

In [ ]:
# print(feature_df[feature_df['label']==1],"   len  =.  ",len(feature_df[feature_df['label']==1]))
feature_df.columns

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, balanced_accuracy_score
# Separate features and labels
# X = feature_df[['dominant_freq', 'mean_amplitude', 'mean_spo2']]
X = feature_df[['EOG LOC-M2_dominant_freq', 'EOG LOC-M2_mean_amplitude',
       'EOG LOC-M2_mean_value', 'EOG ROC-M1_dominant_freq',
       'EOG ROC-M1_mean_amplitude', 'EOG ROC-M1_mean_value',
       'EMG Chin2-Chin1_dominant_freq', 'EMG Chin2-Chin1_mean_amplitude',
       'EMG Chin2-Chin1_mean_value', 'EEG F3-M2_dominant_freq',
       'EEG F3-M2_mean_amplitude', 'EEG F3-M2_mean_value',
       'EEG F4-M1_dominant_freq', 'EEG F4-M1_mean_amplitude',
       'EEG F4-M1_mean_value', 'EEG C3-M2_dominant_freq',
       'EEG C3-M2_mean_amplitude', 'EEG C3-M2_mean_value',
       'EEG C4-M1_dominant_freq', 'EEG C4-M1_mean_amplitude',
       'EEG C4-M1_mean_value', 'EEG O1-M2_dominant_freq',
       'EEG O1-M2_mean_amplitude', 'EEG O1-M2_mean_value',
       'EEG O2-M1_dominant_freq', 'EEG O2-M1_mean_amplitude',
       'EEG O2-M1_mean_value', 'EEG CZ-O1_dominant_freq',
       'EEG CZ-O1_mean_amplitude', 'EEG CZ-O1_mean_value',
       'EMG LLeg-RLeg_dominant_freq', 'EMG LLeg-RLeg_mean_amplitude',
       'EMG LLeg-RLeg_mean_value', 'ECG EKG2-EKG_dominant_freq',
       'ECG EKG2-EKG_mean_amplitude', 'ECG EKG2-EKG_mean_value',
       'Snore_dominant_freq', 'Snore_mean_amplitude', 'Snore_mean_value',
       'Resp PTAF_dominant_freq', 'Resp PTAF_mean_amplitude',
       'Resp PTAF_mean_value', 'Resp Airflow_dominant_freq',
       'Resp Airflow_mean_amplitude', 'Resp Airflow_mean_value',
       'Resp Thoracic_dominant_freq', 'Resp Thoracic_mean_amplitude',
       'Resp Thoracic_mean_value', 'Resp Abdominal_dominant_freq',
       'Resp Abdominal_mean_amplitude', 'Resp Abdominal_mean_value',
       'SpO2_dominant_freq', 'SpO2_mean_amplitude', 'SpO2_mean_value',
       'Rate_dominant_freq', 'Rate_mean_amplitude', 'Rate_mean_value',
       'Capno_dominant_freq', 'Capno_mean_amplitude', 'Capno_mean_value',
       'Resp Rate_dominant_freq', 'Resp Rate_mean_amplitude',
       'Resp Rate_mean_value', 'C-flow_dominant_freq', 'C-flow_mean_amplitude',
       'C-flow_mean_value', 'Tidal Vol_dominant_freq',
       'Tidal Vol_mean_amplitude', 'Tidal Vol_mean_value',
       'Pressure_dominant_freq', 'Pressure_mean_amplitude',
       'Pressure_mean_value']]
y = feature_df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# Initialize the Random Forest Classifier
rfc = RandomForestClassifier(random_state=42)

# Train the model
rfc.fit(X_train, y_train)

# Predict on the test set
y_pred = rfc.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.4f}")

In [ ]:
df.head()

In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import fft

# Assuming your dataset is in a pandas DataFrame called 'df' and it contains the signal columns.

# Define the sampling frequency and segment size
sampling_frequency = 256  # Hz
segment_size = 1280  # 5 seconds of data (1280 rows)

def compute_fft(segment):
    # Ensure the segment is a numpy array
    segment = np.array(segment)
    
    # Check if there are any missing values
    if np.isnan(segment).any():
        segment = np.nan_to_num(segment)  # Replace NaNs with 0
    
    # Apply FFT
    fft_result = fft(segment)
    
    # Calculate power spectrum
    power_spectrum = np.abs(fft_result)**2
    
    return power_spectrum[:len(power_spectrum)//2]  # Only take positive frequencies

# Select only numeric columns for signal data
signal_columns = ['SpO2']  # Modify this based on your actual signal channels

# Initialize an empty list to store the amplitude data (power spectrum)
amplitude_list = []

# Loop through each channel and segment, and apply FFT
for channel in signal_columns:
    channel_amplitudes = []  # List to hold amplitude data for this channel
    for start in range(0, len(df), segment_size):
        segment = df[channel].iloc[start:start + segment_size]
        if len(segment) == segment_size:  # Ensure the segment has the correct size
            power_spectrum = compute_fft(segment)
            channel_amplitudes.append(power_spectrum)
    
    amplitude_list.append(channel_amplitudes)

# Convert the list to a numpy array
# Shape will be (num_channels, num_segments, segment_size/2) since we take only positive frequencies
amplitude_array = np.array(amplitude_list)

# Transpose the array to have shape (num_segments, segment_size/2, num_channels)
amplitude_array = np.transpose(amplitude_array, (1, 2, 0))

# Check the shape of the amplitude array
print("Amplitude array shape:", amplitude_array.shape)

# Now you can use amplitude_array to train the autoencoder
# autoencoder.fit(amplitude_array, amplitude_array, epochs=50, batch_size=32, validation_split=0.2)



# import numpy as np
# import pandas as pd
# from scipy.fft import fft

# # Assuming your dataset is in a pandas DataFrame called 'df' and it contains the signal columns.

# # Define the sampling frequency and segment size
# sampling_frequency = 256  # Hz
# segment_size = 1280  # 5 seconds of data (1280 rows)

# def compute_fft(segment):
#     # Ensure the segment is a numpy array
#     segment = np.array(segment)
    
#     # Debugging information to see the input segment
#     print("Input segment length: ", len(segment))
#     # print("First 5 values of segment: ", segment[:5])
    
#     # Check if there are any missing values
#     if np.isnan(segment).any():
#         print("Warning: Segment contains NaN values. Filling NaNs with 0.")
#         segment = np.nan_to_num(segment)  # Replace NaNs with 0
    
#     # Apply FFT
#     print("Applying FFT...")
#     fft_result = fft(segment)
#     print("FFT result shape: ", fft_result.shape)
    
#     # Calculate power spectrum
#     power_spectrum = np.abs(fft_result)**2
    
#     # Calculate frequencies
#     frequencies = np.fft.fftfreq(len(segment), d=1/sampling_frequency)
    
#     # Debugging information to see the output of FFT and frequencies
#     print("First 5 FFT values: ", fft_result[:5])
#     print("First 5 frequencies: ", frequencies[:5])
    
#     return frequencies, power_spectrum


# # Select only numeric columns for signal data
# signal_columns = df.select_dtypes(include=[np.number]).columns
# print("signal_columns = ",signal_columns)
# signal_columns = ['SpO2']

# # Apply FFT to each channel in the dataset in segments of 1280 rows
# fft_features = []

# # Loop through each channel (assuming channels are in columns)
# for channel in signal_columns:
#     for start in range(0, len(df), segment_size):
#         segment = df[channel].iloc[start:start + segment_size]
#         if len(segment) == segment_size:  # Ensure the segment has the correct size
#             # print("segment = ",segment)
#             print("---------\n")
#             freqs, power = compute_fft(segment)
#             fft_features.append({
#                 'channel': channel,
#                 'frequencies': freqs[:len(freqs)//2],  # Only take positive frequencies
#                 'power_spectrum': power[:len(power)//2]  # Positive frequency components
#             })

# # Convert FFT features into a suitable format for further analysis
# fft_df = pd.DataFrame(fft_features)

# # Display the FFT DataFrame
# print(fft_df)



In [ ]:
# df.columns
amplitude_array[:1].shape

# Pre-processing

## Smoothing each sleep stage df

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Example settings
fs = 256  # Sampling frequency in Hz
epoch_duration = 30  # Each EEG epoch is 30 seconds long
nyquist = 0.5 * fs  # Nyquist frequency

# Frequencies to eliminate (60 Hz and 120 Hz)
freqs_to_eliminate = [60, 120]

# Design a 3rd-order Butterworth bandstop filter
def butter_bandstop(lowcut, highcut, fs, order=3):
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='bandstop')
    return b, a

# Apply the filter to the EEG signal
def apply_bandstop_filter(eeg_signal, fs, freqs_to_eliminate, order=3):
    filtered_signal = eeg_signal.copy()  # Copy to avoid modifying the original
    for freq in freqs_to_eliminate:
        # Create bandstop filters around the target frequencies
        lowcut = freq - 1  # 1 Hz below the target frequency
        highcut = freq + 1  # 1 Hz above the target frequency
        b, a = butter_bandstop(lowcut, highcut, fs, order)
        # Apply filter to the signal
        filtered_signal = signal.filtfilt(b, a, filtered_signal)
    return filtered_signal

# Assuming df is your DataFrame with the 7 EEG columns
eeg_columns = ['EEG F3-M2', 'EEG F4-M1', 'EEG C3-M2', 'EEG C4-M1', 'EEG O1-M2', 'EEG O2-M1', 'EEG CZ-O1']

# Apply the bandstop filter to each EEG column and store the filtered signals
filtered_signals = {}

# Apply the bandstop filter and update the EEG columns in the DataFrame
def update_eeg_columns_with_filtered_signals(df, fs, freqs_to_eliminate, eeg_columns, order=3):
#     filtered_signals = {}
    for column in eeg_columns:
        eeg_signal = df[column].values  # Extract EEG signal from the DataFrame
        filtered_signal = apply_bandstop_filter(eeg_signal, fs, freqs_to_eliminate, order)
        df[column] = filtered_signal  # Update the DataFrame with the filtered signal
    return df

w_df = update_eeg_columns_with_filtered_signals(w_df, fs, freqs_to_eliminate, eeg_columns)
n1_df = update_eeg_columns_with_filtered_signals(n1_df, fs, freqs_to_eliminate, eeg_columns)
n2_df = update_eeg_columns_with_filtered_signals(n2_df, fs, freqs_to_eliminate, eeg_columns)
n3_df = update_eeg_columns_with_filtered_signals(n3_df, fs, freqs_to_eliminate, eeg_columns)
r_df = update_eeg_columns_with_filtered_signals(r_df, fs, freqs_to_eliminate, eeg_columns)

# Plot original vs filtered signal for each column
# plt.figure(figsize=(14, 12))
# for i, column in enumerate(eeg_columns):
#     plt.subplot(len(eeg_columns), 2, 2*i + 1)
#     plt.plot(df.index, df[column])
#     plt.title(f'Original EEG Signal - {column}')
    
#     plt.subplot(len(eeg_columns), 2, 2*i + 2)
#     plt.plot(df.index, filtered_signals[column])
#     plt.title(f'Filtered EEG Signal - {column}')
    
# plt.tight_layout()
# plt.show()



In [ ]:
from scipy.signal import savgol_filter
print(df.columns)
# Define the common window size
# List of columns to smooth (excluding 'anomalies')
columns_to_smooth = n3_df.columns.difference(['anomalies'])

# Apply the moving average filter to each column except 'anomalies'
smoothed_w_df = w_df.copy()  # Create a copy to avoid modifying the original DataFrame
smoothed_n1_df = n1_df.copy()  # Create a copy to avoid modifying the original DataFrame
smoothed_n2_df = n2_df.copy()  # Create a copy to avoid modifying the original DataFrame
smoothed_n3_df = n3_df.copy()  # Create a copy to avoid modifying the original DataFrame
smoothed_r_df = r_df.copy()  # Create a copy to avoid modifying the original DataFrame
# Create a copy to avoid modifying the original DataFrame
common_window_size = 512  # Approximately 2 seconds of data

for col in columns_to_smooth:
    smoothed_w_df[col] = w_df[col].rolling(window=common_window_size, min_periods=1).mean()
    smoothed_n1_df[col] = n1_df[col].rolling(window=common_window_size, min_periods=1).mean()
    smoothed_n2_df[col] = n2_df[col].rolling(window=common_window_size, min_periods=1).mean()
    smoothed_n3_df[col] = n3_df[col].rolling(window=common_window_size, min_periods=1).mean()
    smoothed_r_df[col] = r_df[col].rolling(window=common_window_size, min_periods=1).mean()
    
print(smoothed_w_df.head())  # Check the result
print(smoothed_n1_df.head())  # Check the result
print(smoothed_n2_df.head())  # Check the result
print(smoothed_n3_df.head())  # Check the result
print(smoothed_r_df.head())    # Create a copy to avoid modifying the original DataFrame

In [ ]:
smoothed_w_df.columns,len(smoothed_w_df)

In [ ]:
len(w_df),len(n1_df),len(n2_df),len(n3_df),len(r_df)

In [ ]:
len(smoothed_w_df),len(smoothed_n1_df),len(smoothed_n2_df),len(smoothed_n3_df),len(smoothed_r_df)

## Standardizing/scaling the per sleep stage df

In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd

# Initialize the RobustScaler
scaler = RobustScaler()

# List of DataFrames to scale
# dfs_to_scale = [smoothed_w_df, smoothed_n1_df,smoothed_n3_df, smoothed_r_df]
dfs_to_scale = [smoothed_w_df, smoothed_n1_df, smoothed_n2_df, smoothed_n3_df, smoothed_r_df]

# Perform Robust Scaling on each DataFrame
scaled_dfs = []
for df in dfs_to_scale:
    # Apply scaling (excluding 'anomalies' column, if it's in the DataFrame)
    if 'anomalies' in df.columns:
        features = df.drop(columns=['anomalies'])
        scaled_features = scaler.fit_transform(features)
        scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
        scaled_df['anomalies'] = df['anomalies'].values  # Reattach 'anomalies' column
    else:
        scaled_features = scaler.fit_transform(df)
        scaled_df = pd.DataFrame(scaled_features, columns=df.columns)

    scaled_dfs.append(scaled_df)

# Assign scaled DataFrames to variables
# scaled_w_df, scaled_n1_df, scaled_n3_df, scaled_r_df = scaled_dfs
scaled_w_df, scaled_n1_df, scaled_n2_df, scaled_n3_df, scaled_r_df = scaled_dfs

# Print the heads of the scaled DataFrames
print(scaled_w_df.head())
print(scaled_n1_df.head())
print(scaled_n2_df.head())
print(scaled_n3_df.head())
print(scaled_r_df.head())



In [ ]:
scaled_r_df.columns,len(scaled_r_df.columns),len(scaled_n2_df)

In [ ]:
# scaled_w_df=scaled_w_df.drop(columns=['Patient Event','anomalies'])
# scaled_n1_df=scaled_n1_df.drop(columns=['Patient Event','anomalies'])
# scaled_n2_df=scaled_n2_df.drop(columns=['Patient Event','anomalies'])
# scaled_n3_df=scaled_n3_df.drop(columns=['Patient Event','anomalies'])
# scaled_r_df=scaled_r_df.drop(columns=['Patient Event','anomalies'])

# # scaled_w_df.drop('Patient Event')
# scaled_n3_df=scaled_n3_df.drop(columns=['anomalies'])
# scaled_r_df=scaled_r_df.drop(columns=['anomalies'])
# scaled_w_df=scaled_w_df.drop(columns=['anomalies'])
# scaled_n1_df=scaled_n1_df.drop(columns=['anomalies'])
# scaled_n2_df=scaled_n2_df.drop(columns=['anomalies'])

# scaled_w_df=scaled_w_df.drop(columns=['EtCO2','anomalies'])
# scaled_n1_df=scaled_n1_df.drop(columns=['EtCO2','anomalies'])
# scaled_n2_df=scaled_n2_df.drop(columns=['EtCO2','anomalies'])
# scaled_n3_df=scaled_n3_df.drop(columns=['EtCO2','anomalies'])
# scaled_r_df=scaled_r_df.drop(columns=['EtCO2','anomalies'])

scaled_w_df=scaled_w_df.drop(columns=['Patient Event','EtCO2','anomalies'])
scaled_n1_df=scaled_n1_df.drop(columns=['Patient Event','EtCO2','anomalies'])
scaled_n2_df=scaled_n2_df.drop(columns=['Patient Event','EtCO2','anomalies'])
scaled_n3_df=scaled_n3_df.drop(columns=['Patient Event','EtCO2','anomalies'])
scaled_r_df=scaled_r_df.drop(columns=['Patient Event','EtCO2','anomalies'])

## Valence Inversion Factor

In [ ]:
# Assuming 'df' is your DataFrame
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature in the DataFrame 'df'
vif_data = pd.DataFrame()
vif_data["Feature"] = scaled_r_df.columns
vif_data["VIF"] = [variance_inflation_factor(scaled_r_df.values, i) for i in range(len(scaled_r_df.columns))]

print(vif_data)



## Pair-wise correlation

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame with features
correlation_matrix = scaled_r_df.corr()

# Display the correlation matrix
print(correlation_matrix)

# Optional: To visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Pairwise Correlation Matrix')
plt.show()


## Condition number

In [ ]:
import numpy as np

# Assuming 'df' is your DataFrame with features
X = scaled_r_df.values  # Convert the DataFrame to a NumPy array

# Calculate the condition number using the singular values
condition_number = np.linalg.cond(X)

print(f"Condition Number: {condition_number}")

## PCA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Function to perform PCA and plot the explained variance ratio (elbow plot)
def plot_pca_explained_variance(df):
    """
    This function performs PCA on the DataFrame and plots the cumulative explained variance 
    to help determine the optimal number of components using the elbow method.
    
    :param df: Input DataFrame.
    :return: PCA model (sklearn PCA object) for further use.
    """
    # Apply PCA with as many components as there are features
    pca = PCA(n_components=min(df.shape[0], df.shape[1]))
    pca.fit(df)
    
    # Calculate the cumulative explained variance
    cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)
    
    # Plot the elbow graph
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
    plt.title('Explained Variance by Number of PCA Components')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')  # Optional 95% line
    plt.legend()
    plt.grid()
    plt.show()
    
    return pca

# Apply to your DataFrames (example: scaled_w_df)
# pca_model_w = plot_pca_explained_variance(scaled_w_df)
# pca_model_n1 = plot_pca_explained_variance(scaled_n1_df)
# pca_model_n2 = plot_pca_explained_variance(scaled_n2_df)
# pca_model_n3 = plot_pca_explained_variance(scaled_n3_df)
pca_model_r = plot_pca_explained_variance(scaled_r_df)


In [ ]:
# Assuming you've used sklearn's PCA
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# Let's assume 'data' is your dataset and 'pca' is the fitted PCA model
# Perform PCA on your dataset
pca = PCA()
pca.fit(scaled_r_df)

# Get the explained variance ratios
explained_variance_ratios = pca.explained_variance_ratio_

# Get the cumulative explained variance
cumulative_variance = np.cumsum(explained_variance_ratios)

# Find out how many components are needed to explain 96% variance
n_components_96 = np.argmax(cumulative_variance >= 0.96) + 1
print(f"Number of components to reach 96% variance: {n_components_96}")

# Get the PCA components (loadings)
loadings = pca.components_

# Create a DataFrame with the features and their contributions to each principal component
# Assuming 'feature_names' is the list of original feature names in your dataset
loadings_df = pd.DataFrame(loadings[:n_components_96], columns=scaled_r_df.columns)

# Print the contributions of features for the selected components
print(loadings_df)

# Calculate the magnitude of the loadings
loadings_df['magnitude'] = loadings_df.abs().max(axis=1)

# Sort the loadings dataframe by magnitude in descending order
sorted_loadings_df = loadings_df.sort_values(by='magnitude', ascending=False)

# Drop the magnitude column if you want to remove it from the output
sorted_loadings_df = sorted_loadings_df.drop('magnitude', axis=1)

# Print the sorted loadings dataframe
for i in sorted_loadings_df:
    print(i)
# print(sorted_loadings_df)



In [ ]:
import pandas as pd

# Set pandas to display all columns
pd.set_option('display.max_columns', None)

# Calculate the magnitude of the loadings
loadings_df['magnitude'] = loadings_df.abs().max(axis=1)

# Sort the loadings dataframe by magnitude in descending order
sorted_loadings_df = loadings_df.sort_values(by='magnitude', ascending=False)

# Drop the magnitude column if you don't want to display it
sorted_loadings_df = sorted_loadings_df.drop('magnitude', axis=1)

# Print the sorted loadings dataframe
print(sorted_loadings_df)

In [ ]:
sorted_loadings_df.columns,sorted_loadings_df.shape

# RFC (partially annotated dataset)

In [ ]:
# Calculate the number of rows representing the first 20% of the DataFrame
percent = 0.30
percent_index = int(percent * len(wavelet_features_df))
print(percent_index)
# Slice the first 20% of the DataFrame
first_percent_df = wavelet_features_df.iloc[:percent_index]

# Count the number of anomalies in the first x% of the DataFrame
anomaly_count_first_percent = (first_percent_df['anomalies'] == 1).sum()

print(f"Number of anomalies in the first {percent*100}% of the DataFrame:", anomaly_count_first_percent)

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report

sleep_stage_dfs = [scaled_w_df,scaled_n1_df,scaled_n2_df,scaled_n3_df,scaled_r_df]

for test_df in sleep_stage_dfs:
    # n = min(n,len(non_anomalies))

    X = test_df.drop(columns=['anomalies'])
    y = test_df['anomalies']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=42)
    
    '''
    # Select every 20th row for training
    train_indices = X.index[::20]  # Get every 20th row index
    test_indices = X.index.difference(train_indices)  # Remaining indices for testing
    
    # Create train and test sets
    X_train = X.loc[train_indices]
    y_train = y.loc[train_indices]
    X_test = X.loc[test_indices]
    y_test = y.loc[test_indices]
    '''
    
    # Get the row indices of X_train
    train_indices = X_train.index
    
    # Print the row indices
    # print("Row indices in X_train:", train_indices.tolist())
    
    print(len(X_train),len(X_test),len(y_train),len(y_test))
    print("TRAIN SET: ", (y_train == 1).sum(), (y_train == 0).sum())
    print("TEST SET: ", (y_test == 1).sum(), (y_test == 0).sum())

    # Initialize the Random Forest Classifier
    rfc = RandomForestClassifier(random_state=42)

    # Train the model
    rfc.fit(X_train, y_train)
    # rfc.fit(X_train_balanced, y_train_balanced)

    # Predict on the test set
    y_pred = rfc.predict(X_test)

    # Calculate balanced accuracy
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    acc_list.append(balanced_acc)
    print(f"Balanced Accuracy for n={n}: {balanced_acc:.4f}")

    # Optional: Classification report
    print(f"Classification Report for n={n}:\n", classification_report(y_test, y_pred))
    print("="*50)